In [26]:
import pylab as plt
import tensorflow_probability.substrates.jax as tfp
from jax import numpy as jnp
from jax import random

from jaxns import ExactNestedSampler, Prior, Model
from jaxns import TerminationCondition
from jaxns import analytic_log_evidence

tfpd = tfp.distributions

In [37]:
import tinygp

def get_kernel_params(kernel_type):
    if kernel_type == "RN":
        return ['arn','crn']

def get_mean_params(mean_type):
    if mean_type == "gaussian":
        return ['A', 't0', 'sig'] 

def get_gp_params(kernel_type, mean_type):
    kernel_params = get_kernel_params(kernel_type)
    mean_params = get_mean_params(mean_type)
    kernel_params.extend(mean_params)
    return kernel_params

print(get_kernel_params("RN"), get_mean_params("gaussian"))
print(get_gp_params("RN", "gaussian"))

params_list = get_gp_params("RN", "gaussian")

def get_prior(params_list, **kwargs):

    def prior_model():
        prior_list = []
        for i in params_list:
            parameter = yield Prior (kwargs[i], name = i)
            prior_list.append(parameter)
        return tuple(prior_list)
    return prior_model
    pass

prior_dict = {"arn" : tfpd.Uniform(0.1, 2),"t0" : tfpd.Uniform(- 0.1,0.1 ) , "sig" : tfpd.Uniform(0.5 , 2),
              "crn": tfpd.Uniform(0, 3), "A" : tfpd.Uniform(0.1 , 2 ),
              }

exp_prior = get_prior( params_list, **prior_dict)
print(exp_prior)
print(exp_prior())

for i in exp_prior():
    print(i)

# def RNlog_likelihood(arn, crn, A, t0, sig):

def get_likelihood(params_list, **kwargs):
    def likelihood_model(*args):    
        dict = {}
        for i, params in enumerate(params_list):
            dict[params] = args[i]
        print(dict)

        # cannot make dicts like this
        rnlikelihood_params = {
            "arn": dict["arn"],
            "crn": dict["crn"],
            "aqpo": 0.0,
            "cqpo": 0.0,
            "freq": 0.0,
        }

        mean_params = {
            "A": dict["A"],
            "t0": dict["t0"],
            "sig": dict["sig"],
        }
        # kernel = get_kernel(kernel_type="RN", kernel_params=rnlikelihood_params)
        # mean = get_mean(mean_type=mean_type, mean_params=mean_params)
        # gp = GaussianProcess(kernel, kwargs["Times"], mean_value=mean(kwargs["Times"]))
        # return gp.log_probability(kwargs["counts"])
        return -1.5
    return likelihood_model

likelihood_model = get_likelihood(params_list, **prior_dict)

# model = Model(prior_model= exp_prior, log_likelihood= RNlog_likelihood)
model = Model(prior_model= exp_prior, log_likelihood= likelihood_model)
model.sanity_check(random.PRNGKey(10), S=100)


INFO[2023-06-20 20:47:59,854]: Sanity check...
INFO[2023-06-20 20:47:59,859]: Sanity check passed


['arn', 'crn'] ['A', 't0', 'sig']
['arn', 'crn', 'A', 't0', 'sig']
<function get_prior.<locals>.prior_model at 0x15a5e4af0>
<generator object get_prior.<locals>.prior_model at 0x15a432970>
arn	() -> () <class 'numpy.float32'>
crn	() -> () <class 'numpy.float32'>
A	() -> () <class 'numpy.float32'>
t0	() -> () <class 'numpy.float32'>
sig	() -> () <class 'numpy.float32'>
{'arn': Traced<ShapedArray(float32[])>with<BatchTrace(level=2/0)> with
  val = Traced<ShapedArray(float32[100])>with<DynamicJaxprTrace(level=1/0)>
  batch_dim = 0, 'crn': Traced<ShapedArray(float32[])>with<BatchTrace(level=2/0)> with
  val = Traced<ShapedArray(float32[100])>with<DynamicJaxprTrace(level=1/0)>
  batch_dim = 0, 'A': Traced<ShapedArray(float32[])>with<BatchTrace(level=2/0)> with
  val = Traced<ShapedArray(float32[100])>with<DynamicJaxprTrace(level=1/0)>
  batch_dim = 0, 't0': Traced<ShapedArray(float32[])>with<BatchTrace(level=2/0)> with
  val = Traced<ShapedArray(float32[100])>with<DynamicJaxprTrace(level=1/